In [1]:
# read in packages
%matplotlib inline
import pandas as pd
import json
import geopandas as gpd
import numpy as np
from shapely import wkt
import os
from keplergl import KeplerGl

In [4]:
# allow max rows and colums to be displayed
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
os.chdir(r'C:\Users\steve\GitHub\rp-covid-migration')

In [31]:
df_centroid = pd.read_csv(r'data/values_cntr_janaug.csv')
def df_kepler(df):
    df_centroid['datetime'] = pd.to_datetime(df_centroid['variable'])
    df_centroid['datetime'] =df_centroid["datetime"].dt.strftime('%m/%d/%Y')
    df_centroid['datetime'] = df_centroid['datetime'].astype(str) + ' 0:00'
    #df_centroid.drop(columns = 'geometry')
    df_centroid['geom'] = df_centroid['geom'].apply(wkt.loads)
    df_centroid['value'] = df_centroid['value'].astype(float)
    #df_centroid = gpd.GeoDataFrame(df_centroid, geometry='geom')
    df_centroid.to_csv(r'data/kepler_subplaces_dates.csv', index = False)


In [32]:
df_kepler(df_centroid)

In [13]:
config = {
'version': 'v1',
'config': {
    'mapState': {
        'latitude': 40.7137,
        'longitude': -73.9675,
        'zoom': 12.3
    }}}

In [14]:
map1 = KeplerGl(height = 300)
map1.add_data(data=df_centroid,name ='df')
map1.config = config

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


ValueError: Can't clean for JSON: <shapely.geometry.point.Point object at 0x000002908F9D4208>